In [19]:
from matplotlib import pyplot as plt
import os
import re
import cv2
from skimage.io import imsave
import numpy as np
import nibabel as nib

In [20]:
fnames = [
    'rec_8bit_abs_00002_rot_crop_513x1737x1716.raw',
    'rec_8bit_abs_00012_rot_crop_487x1766x1716.raw',
    'rec_8bit_abs_00018_rot_crop_487x1771x1716.raw',
    'rec_8bit_abs_00024_rot_crop_490x1771x1716.raw',
    'rec_8bit_abs_00030_rot_crop_488x1813x1716.raw',
    'rec_8bit_abs_00036_rot_crop_483x1789x1716.raw',
    'rec_8bit_abs_00042_rot_crop_483x1819x1716.raw',
    'rec_8bit_abs_00045_rot_crop_482x1838x1716.raw',
    'rec_8bit_abs_00048_rot_crop_498x1792x1716.raw',
    'rec_8bit_abs_00051_rot_crop_487x1775x1716.raw',
    'rec_8bit_abs_00053_rot_crop_494x1775x1716.raw',
    'rec_8bit_abs_00054_rot_crop_481x1785x1716.raw',
    'rec_8bit_abs_00055_rot_crop_492x1775x1716.raw',
    'rec_8bit_abs_00056_rot_crop_471x1781x1716.raw',
    'rec_8bit_abs_00058_rot_crop_487x1777x1716.raw',
    'rec_8bit_abs_00059_rot_crop_483x1775x1716.raw',
    'rec_8bit_abs_00060_rot_crop_478x1766x1716.raw'
]

In [21]:
crp_pos = {
    fnames[-17]:[455,1315,45,455],
    fnames[-16]:[510,1375,35,445],
    fnames[-15]:[525,1400,35,445],
    fnames[-14]:[540,1415,35,445],
    fnames[-13]:[595,1475,35,445],
    fnames[-12]:[590,1475,30,445],
    fnames[-11]:[625,1515,25,445],
    fnames[-10]:[650,1540,35,455],
    fnames[-9]:[595,1490,40,460],
    fnames[-8]:[570,1490,30,450],
    fnames[-7]:[570,1490,35,450],
    fnames[-6]:[570,1490,35,450],
    fnames[-5]:[555,1475,35,450],
    fnames[-4]:[555,1490,25,450],
    fnames[-3]:[555,1490,30,460],
    fnames[-2]:[555,1490,30,460],
    fnames[-1]:[560,1505,40,460]
}

In [22]:
def get_size(file_name):
    matches = re.findall(r'\d+', file_name)
    matches = [int(match) for match in matches]
    force = matches[1]
    length = matches[2]
    width = matches[3]
    height = matches[4]

    return length, width, height

In [31]:
for i in range(len(fnames)):
    file_name = fnames[i]
    length, width, height = get_size(file_name) # length:400-500, width:1700-1800(perpendicular to fiber direction), height:1717(parallel to fiber direction)
    im = np.fromfile('./dataset/' + file_name, dtype=np.uint8)
    im = im.reshape((height,width,length))

    
    im_slice = im[int(height/2)]
    new_w = crp_pos[file_name][1] - crp_pos[file_name][0]
    im_slice = im[int(height/2)].copy()
    im_slice[crp_pos[file_name][0]+int(0.07*new_w), :] = 0 
    im_slice[crp_pos[file_name][1]-int(0.13*new_w), :] = 0 
    im_slice[:, crp_pos[file_name][2]] = 0
    im_slice[:, crp_pos[file_name][3]] = 0

    imsave(os.path.join('cropped_data/slices', str(i).zfill(5) + '.png'),
                im_slice)
    

    cropped_img = im[:,
                     crp_pos[file_name][0]+int(0.07*new_w) : crp_pos[file_name][1]-int(0.13*new_w),
                     crp_pos[file_name][2] : crp_pos[file_name][3]]
    nifti_img = nib.Nifti1Image(cropped_img, np.eye(4))
    new_name = str(i).zfill(5) + '_1716x' + str(cropped_img.shape[1]) + "x" + str(cropped_img.shape[2]) + '.nii.gz'
    nib.save(nifti_img, os.path.join('cropped_data/data', new_name))
